In [ ]:
from langchain_aws import ChatBedrock
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import boto3

In [ ]:
# looks up the id for the guardrail named 'sanitize'
bedrock_client = boto3.client('bedrock')
guardrails = bedrock_client.list_guardrails()
this_guardrail_id = None
for guardrail in guardrails['guardrails']:
    if guardrail['name'] == 'sanitizer':
        this_guardrail_id = guardrail['id']
        break
print("guardrail ID: " + this_guardrail_id)

# reference to the foundation model via amazon bedrock.
# https://docs.aws.amazon.com/bedrock/latest/userguide/inference-parameters.html
model = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # claude is awesome!    
    model_kwargs={
        "temperature": 0.95,
        # "top_p": 0.95,
        # "top_k": 250,
        "max_tokens": 20000
    },
    guardrails={                                         # guardrails filter prompts and responses
        "guardrailIdentifier": this_guardrail_id, 
        "guardrailVersion": "1"
    }
)

In [ ]:
# there are no experts!
prompt = PromptTemplate(
    template="""
Imagine three different experts are answering this question.

All experts will write down 1 step of their thinking,
then share it with the group.

Then all experts will go on to the next step, etc.

If any expert realises they're wrong at any point then they leave.

The question is... what is 2*13-23*1(2%2)?
"""
)

In [ ]:
chain = prompt | model | StrOutputParser()

In [ ]:
print(chain.invoke({}))